In [1]:
import pandas as pd
from predictables.core.src._UnivariateAnalysis import UnivariateAnalysis
from predictables.util import get_column_dtype


# load data
df = pd.read_parquet("./credit_risk.parquet")
# df.columns = [c.replace(" ", "_") for c in df.columns]
# df["cv"] = np.random.randint(1, 11, df.shape[0])
# objs, d = [], []
# for col in tqdm(df.columns.tolist()[:-2]):
#     if get_column_dtype(df[col]) == "continuous":
#         df[col] = df[col].astype(float)
#     elif get_column_dtype(df[col]) == "binary":
#         df[col] = df[col].astype(float).astype(int).astype(str).astype("category")
#     elif get_column_dtype(df[col]) == "categorical":
#         try:
#             df[col] = df[col].astype(float).astype(str).astype("category")
#         except Exception:
#             df[col] = df[col].astype(str).astype("category")
#     elif get_column_dtype(df[col]) == "date":
#         df[col] = pd.to_datetime(df[col])
#         d.append(col)
#     else:
#         df[col] = df[col].astype(str)
#         objs.append(col)

# # put small amount of noise on the target variable - to help with numerical stability
# # when fitting the model
# df["noise"] = stats.norm.rvs(loc=0, scale=0.01, size=df.shape[0])
# df["abs"] = np.abs(df["noise"])
# df["perturbed_target"] = (
#     df["TARGET"] + (df["abs"] * df["TARGET"].eq(0)) - (df["abs"] * df["TARGET"].eq(1))
# )
df.to_parquet("./credit_risk.parquet")
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,ORGANIZATION_TYPE_Security_Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Transport:_type_3,ORGANIZATION_TYPE_Transport:_type_4,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,TARGET,cv,perturbed_target
0,194347.0,0,1,0,0.105263,0.001512,0.170652,0.218673,0.158249,0.429796,...,0,0,0,0,0,0,1,1.0,6,0.994695
1,220267.0,0,0,1,0.000000,0.001897,0.154330,0.077624,0.127946,0.429796,...,0,0,0,0,0,0,0,0.0,6,0.011183
2,173889.0,1,1,0,0.000000,0.002474,0.157303,0.125325,0.158249,0.635991,...,0,0,0,0,0,0,0,0.0,4,0.000548
3,297077.0,0,0,1,0.000000,0.001704,0.266766,0.187908,0.225589,0.256321,...,0,0,0,0,0,0,0,0.0,8,0.022819
4,174102.0,0,0,1,0.105263,0.000742,0.128303,0.166409,0.108866,0.245631,...,0,0,0,0,0,0,0,0.0,10,0.003569


In [2]:
# df = pd.read_parquet("./cancerdf.parquet")
# df.head()

In [3]:
df1 = df[df.columns.tolist()[:10] + df.columns.tolist()[-10:]]
df1 = df1.drop(
    columns=[
        col
        for col in df1.columns
        if get_column_dtype(df1[col]) in ["date", "categorical", "binary"]
    ]
)
df1["cv"] = df["cv"]
df1["TARGET"] = df["TARGET"]
df1.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,perturbed_target,cv,TARGET
0,194347.0,0.105263,0.001512,0.170652,0.218673,0.158249,0.429796,0.994695,6,1.0
1,220267.0,0.000000,0.001897,0.154330,0.077624,0.127946,0.429796,0.011183,6,0.0
2,173889.0,0.000000,0.002474,0.157303,0.125325,0.158249,0.635991,0.000548,4,0.0
3,297077.0,0.000000,0.001704,0.266766,0.187908,0.225589,0.256321,0.022819,8,0.0
4,174102.0,0.105263,0.000742,0.128303,0.166409,0.108866,0.245631,0.003569,10,0.0


In [4]:
df1.TARGET.value_counts()

TARGET
0.0    27505
1.0     2495
Name: count, dtype: int64

In [5]:
df1.shape

(30000, 10)

In [6]:
from sklearn.model_selection import train_test_split

dft, dfv = train_test_split(df1, test_size=0.2, random_state=42, stratify=df1["TARGET"])
ua = UnivariateAnalysis(
    model_name="Credit Risk Model",
    df_train=dft,
    df_val=dfv,
    target_column_name="TARGET",
    feature_column_names=df1.columns.tolist()[:-3],
    cv_column_name="cv",
    has_time_series_structure=False,
)

Performing univariate analysis on 7 features:   0%|          | 0/7 [00:00<?, ?it/s]

Performing univariate analysis on 7 features:  29%|██▊       | 2/7 [01:42<04:15, 51.09s/it]/home/aweaver/work/predictables/.venv/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/aweaver/work/predictables/.venv/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
Performing univariate analysis on 7 features: 100%|██████████| 7/7 [05:57<00:00, 51.03s/it]


In [21]:
ua.build_report("credit.pdf", max_per_file=20)

TypeError: df must be a pandas or polars dataframe. Got <class 'str'>.

In [22]:
ua._sort_features_by_ua()

TypeError: df must be a pandas or polars dataframe. Got <class 'str'>.

### fit each fold to a model

In [6]:
X, y = (
    df.collect().to_pandas().drop(columns="target"),
    df.select("target").collect().to_pandas()["target"],
)
X.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,5
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,3
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [7]:
df = df.collect().to_pandas()
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,5
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,3
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,1


In [8]:
from predictables.univariate.src._fit_sm_logistic_regression import (
    fit_sm_logistic_regression,
)
from predictables.core._PredicTables import PredicTables
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

fit_sm_logistic_regression(X_train.iloc[:, :3], y_train)

In [9]:
pt = PredicTables(
    "Cancer Model",
    pd.concat([X_train.iloc[:, :3], y_train], axis=1),
    pd.concat([X_test.iloc[:, :3], y_test], axis=1),
    pd.concat([X_test.iloc[:, :3], y_test], axis=1),
    "target",
    X_train["cv"],
    False,
)
pt.ua.build_report("new_test_report.pdf", max_per_file=5)

TypeError: UnivariateAnalysis.__init__() got multiple values for argument 'has_time_series_structure'

In [ ]:
pt.ua._sort_features_by_ua().index.tolist()

Sorting features by their relevance based on an univariate analysis: 100%|██████████| 3/3 [00:00<00:00, 17403.75it/s]


['mean_texture', 'mean_radius', 'mean_perimeter']